In [3]:
import numpy as np
x_train=np.load('/storage/ADHD200/athena/NYU/All_Data.npy')

In [ ]:
'''This script demonstrates how to build a variational autoencoder with Keras.

 #Reference

 - Auto-Encoding Variational Bayes
   https://arxiv.org/abs/1312.6114
'''
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.datasets import mnist

batch_size = 1
original_dim=x_train.shape[1]
latent_dim = 32
intermediate_dim = 500
epochs = 10
epsilon_std = 1.0


x = Input(shape=(original_dim,))
h = Dense(256, activation='tanh')(x)
h = Dense(128, activation='tanh')(h)
h = Dense(64, activation='tanh')(h)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)


def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# we instantiate these layers separately so as to reuse them later

h_decoded = Dense(64, activation='tanh')(z)
h_decoded = Dense(128, activation='tanh')(h_decoded)
h_decoded = Dense(256, activation='tanh')(h_decoded)
x_decoded_mean = Dense(original_dim, activation='sigmoid')(h_decoded)

# instantiate VAE model
vae = Model(x, x_decoded_mean)

# Compute VAE loss
xent_loss = original_dim * metrics.binary_crossentropy(x,x_decoded_mean )
kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae_loss = K.mean(xent_loss + kl_loss)

vae.add_loss(vae_loss)
from keras import optimizers
rmsprop=optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.001)
vae.compile(optimizer=rmsprop)
vae.summary()


vae.fit(x_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size)

encoder = Model(x, z_mean)



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 28546)        0                                            
__________________________________________________________________________________________________
dense_28 (Dense)                (None, 256)          7308032     input_4[0][0]                    
__________________________________________________________________________________________________
dense_29 (Dense)                (None, 128)          32896       dense_28[0][0]                   
__________________________________________________________________________________________________
dense_30 (Dense)                (None, 64)           8256        dense_29[0][0]                   
__________________________________________________________________________________________________
dense_31 (

In [ ]:
decoder=Model(x, z)
y=decoder.predict(x_train,
        batch_size=batch_size)
y.shape

In [ ]:

import numpy as np
t = range(0, 200)

import matplotlib.pyplot as plt
plt.plot(t, y[:,1])

In [ ]:
from sklearn.linear_model import Lasso
clf = Lasso(alpha=0.1)
clf.fit(y,x_train)
print(clf.coef_.shape)

In [ ]:
#W=decoder_mean.get_weights()
#components_img=W[0]


In [ ]:
from nilearn.input_data import NiftiMasker
masker = NiftiMasker(mask_img=canica.masker_.mask_img_, standardize=True)
masker.fit()

import numpy as np
components_img = masker.inverse_transform(np.transpose(clf.coef_))

In [ ]:
from nilearn import plotting

plotting.plot_prob_atlas(components_img, view_type='filled_contours',
                         title='All RBM components')

In [ ]:
from nilearn.image import iter_img
from nilearn.plotting import plot_stat_map, show

for i, cur_img in enumerate(iter_img(components_img)):
    plot_stat_map(cur_img, display_mode="ortho", title="IC %d" % i,
                  cut_coords=None, colorbar='bwr')

show()